In [1]:
import warnings
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
import xgboost as xgb
from collections import defaultdict

warnings.filterwarnings('ignore')

SEED = 57

In [2]:
df_tv = pd.read_csv('data/train_preprocessed_fold.csv', index_col='PassengerId')
df_tt = pd.read_csv('data/test_preprocessed.csv', index_col='PassengerId')

In [3]:
df_tt.columns

Index(['Pclass_2nd', 'Pclass_3rd', 'Sex_male', 'Fare_low', 'Fare_medium',
       'Fare_v_high', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F',
       'Cabin_G', 'Cabin_T', 'Embarked_Q', 'Embarked_S', 'Title_Miss',
       'Title_Mr', 'Title_Mrs', 'Title_Others', 'Family_Size_large',
       'Family_Size_medium', 'Family_Size_small', 'Age_Group_elder',
       'Age_Group_kid', 'Age_Group_teen'],
      dtype='object')

In [4]:
traning_cols = ['Pclass_2nd', 'Pclass_3rd', 'Sex_male', 'Fare_low', 'Fare_medium', 'Fare_v_high', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T', 'Embarked_Q', 'Embarked_S', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Others', 'Family_Size_large', 'Family_Size_medium', 'Family_Size_small', 'Age_Group_elder', 'Age_Group_kid', 'Age_Group_teen']
target_col = 'Survived'

In [5]:
classifiers = {
    'lr': linear_model.LogisticRegression,
    'svc': svm.SVC,
    'lsvc': svm.LinearSVC,
    'gnb': naive_bayes.GaussianNB,
    'mnb': naive_bayes.MultinomialNB,
    'knn': neighbors.KNeighborsClassifier,
    'dt': tree.DecisionTreeClassifier,
    'rf': ensemble.RandomForestClassifier,
    'ada': ensemble.AdaBoostClassifier,
    'gb': ensemble.GradientBoostingClassifier,
    'sgd': linear_model.SGDClassifier,
    'xgb': xgb.XGBClassifier
}

classifier_params = {
    'lr': {
        'params': {'C': 1000.0, 'random_state': 57},
        'score': 0.829
    },
    'svc': {
        'params': {'C': 1000.0, 'gamma': 0.01, 'random_state': 57},
        'score': 0.830
    },
    'lsvc': {
        'params': {'C': 1.0, 'random_state': 57},
        'score': 0.827
    },
    'gnb': {
        'params': {'var_smoothing': 1e-06},
        'score': 0.757
    },
    'mnb': {
        'params': {'alpha': 0.8},
        'score': 0.781
    },
    'knn': {
        'params': {'n_neighbors': 7},
        'score': 0.791
    },
    'dt': {
        'params': {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 0.1, 'random_state': 57},
        'score': 0.775
    },
    'rf': {
        'params': {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 0.1, 'n_estimators': 200, 'random_state': 57},
        'score': 0.787
    },
    'ada': {
        'params': {'n_estimators': 100, 'random_state': 57},
        'score': 0.817
    },
    'gb': {
        'params': {'min_samples_leaf': 0.1, 'n_estimators': 150, 'random_state': 57},
        'score': 0.836
    },
    'sgd': {
        'params': {'alpha': 0.001, 'penalty': 'elasticnet', 'random_state': 57},
        'score': 0.824
    },
    'xgb': {
        'params': {'eta': 1.0, 'gamma': 1.0, 'lambda': 10.0, 'seed': 57},
        'score': 0.839
    }
}

In [6]:
def generate_meta_feature(classifier_name, classifier, params, fold):
    df_tr = df_tv[df_tv.kfold != fold]
    df_vl = df_tv[df_tv.kfold == fold]

    xtr, ytr = df_tr[traning_cols], df_tr[target_col]
    xvl = df_tr[traning_cols]

    ypd = classifier(**params).fit(xtr, ytr).predict(xvl)

    df_tv.loc[xvl.index, classifier_name] = ypd

In [7]:
for classifier_name, classifier in classifiers.items():
    params = classifier_params[classifier_name]['params']
    df_tv[classifier_name] = None
    for fold in range(5): generate_meta_feature(classifier_name, classifier, params, fold)

In [8]:
df_tv.head()

,Survived,Pclass_2nd,Pclass_3rd,Sex_male,Fare_low,Fare_medium,Fare_v_high,Cabin_B,Cabin_C,Cabin_D,...,lsvc,gnb,mnb,knn,dt,rf,ada,gb,sgd,xgb
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1,1,1,1,1,1,1,1,1,1
3,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1,1,1,1,1,1,1,0,1,0
4,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1,1,1,1,1,1,1,1,1,1
5,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
xtr, ytr = df_tv[traning_cols], df_tv[target_col]
for classifier_name, classifier in classifiers.items():
    params = classifier_params[classifier_name]['params']
    ypd = classifier(**params).fit(xtr, ytr).predict(df_tt[traning_cols])
    df_tt[classifier_name] = ypd

In [10]:
df_tt.head(5)

,Pclass_2nd,Pclass_3rd,Sex_male,Fare_low,Fare_medium,Fare_v_high,Cabin_B,Cabin_C,Cabin_D,Cabin_E,...,lsvc,gnb,mnb,knn,dt,rf,ada,gb,sgd,xgb
PassengerId,,,,,,,,,,,,,,,,,,,,,
892,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
893,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1,1,1,0,0,0,1,0,1,0
894,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
895,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
896,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1,1,1,1,0,1,1,1,1,1


In [11]:
df_tv.to_csv('data/train_preprocessed_meta.csv')
df_tt.to_csv('data/test_preprocessed_meta.csv')